# Imports

In [1]:
import os
import sys
import requests

In [2]:
import pandas as pd
import geopandas as gpd

import from local module

In [3]:
proj_folder = os.path.dirname(os.getcwd())
module_folder = "api"
import_path = os.path.join(proj_folder, module_folder)
print(import_path)
sys.path.append(import_path)

/home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api


In [4]:
#These geo imports will be available from within the same file: api/geo.py
#import func to get locally stored file with uk regions
from geo import get_regional_geojson
#carbon imports for later
from geo import get_api_carbon_regional_response

## Load in the local geojson

The geography of each region is stored in this variable. Also all id columns. This is what the carbon and solar data will be merged with

In [6]:
#show the base data, to attach solar + carbon data to
local_reg_geojson = get_regional_geojson().sort_values(by="ID")
local_reg_geojson

loading uk energy regions from file /home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api/../data/geo/uk_dno_regions_2024_lonlat.geojson


,ID,Name,DNO,Region,DNO_Full,pes_id,geometry
0,1,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,17,"MULTIPOLYGON (((-5.1162 55.2412, -5.11948 55.2..."
1,2,_N,SPEN,South Scotland,SP Energy Networks,18,"MULTIPOLYGON (((-4.7016 54.66506, -4.70051 54...."
2,3,_G,ENWL,North West England,Electricity North West,16,"MULTIPOLYGON (((-2.66161 55.14636, -2.65693 55..."
3,4,_F,NPG,North East England,Northern Powergrid,15,"MULTIPOLYGON (((-1.59801 55.62683, -1.59862 55..."
4,5,_M,NPG,Yorkshire,Northern Powergrid,23,"MULTIPOLYGON (((-2.22088 54.25104, -2.21629 54..."
5,6,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,13,"MULTIPOLYGON (((-4.255 53.38794, -4.25559 53.3..."
6,7,_K,NGED,South Wales,National Grid Electricity Distribution,21,"MULTIPOLYGON (((-3.11356 51.3757, -3.11456 51...."
7,8,_E,NGED,West Midlands,National Grid Electricity Distribution,14,"MULTIPOLYGON (((-1.959 53.21662, -1.95676 53.2..."
8,9,_B,NGED,East Midlands,National Grid Electricity Distribution,11,"MULTIPOLYGON (((0.26687 52.8124, 0.26697 52.81..."
9,10,_A,UKPN,East England,UK Power Networks,10,"MULTIPOLYGON (((0.8747 51.51298, 0.87653 51.51..."


## Functions that call the carbon and solar api's, and return their data dfs respectively

This means getting the live data, without the geographical part attached yet, just region ids. The region ids are not as simple as you'd hope, so refer to local geodf (see above)

In [7]:
#was already in geo but adding here for completeness
def carbon_intensity_live_df():
    """
        Orchestrates the API call and coversion to df functions
    """
    regions = get_api_carbon_regional_response()

    def carbon_regional_response_to_df(regions):
        """
            parse API response into desired df, with columns:
            'id', 'dno_region', 'api_name', 'intensity_forecast', 'intensity_index',
        'biomass_perc', 'coal_perc', 'imports_perc', 'gas_perc', 'nuclear_perc',
        'other_perc', 'hydro_perc', 'solar_perc', 'wind_perc'
        """
        print("Converting carbon intensity api response to df...")
        rows = []
        for region in regions:
            region_id = region["regionid"]
            dno_region = region["dnoregion"]
            name = region["shortname"]
            intensity_forecast = region["intensity"]["forecast"]
            intensity_index = region["intensity"]["index"]

            time_from, time_to = region['times']['time_from'], region['times']['time_to']
            
            row_dict = {
                "carbon_region_id" : region_id,
                "dno_region" : dno_region,
                "api_name" : name,
                "carbon_intensity_forecast" : intensity_forecast,
                "carbon_intensity_index" : intensity_index,
                "carbon_time_from" : time_from,
                "carbon_time_to" : time_to
            }

            for fuel in region["generationmix"]:
                fuel_name = fuel["fuel"]
                fuel_percentage = fuel["perc"]
                row_dict[fuel_name+"_perc"] = fuel_percentage

            rows.append(row_dict)

        return pd.DataFrame(rows)

    regions_df = carbon_regional_response_to_df(regions)

    return regions_df
carbon_df = carbon_intensity_live_df()
carbon_df

Getting current regional carbon intensity from:  https://api.carbonintensity.org.uk/regional
Converting carbon intensity api response to df...


,carbon_region_id,dno_region,api_name,carbon_intensity_forecast,carbon_intensity_index,carbon_time_from,carbon_time_to,biomass_perc,coal_perc,imports_perc,gas_perc,nuclear_perc,other_perc,hydro_perc,solar_perc,wind_perc
0,1,Scottish Hydro Electric Power Distribution,North Scotland,0,very low,2024-07-10T22:30Z,2024-07-10T23:00Z,0.0,0,0.0,0.0,0.0,0,1.7,0,98.3
1,2,SP Distribution,South Scotland,18,very low,2024-07-10T22:30Z,2024-07-10T23:00Z,2.4,0,0.0,3.9,29.0,0,0.9,0,63.8
2,3,Electricity North West,North West England,25,very low,2024-07-10T22:30Z,2024-07-10T23:00Z,2.7,0,10.7,5.4,55.5,0,0.2,0,25.5
3,4,NPG North East,North East England,18,very low,2024-07-10T22:30Z,2024-07-10T23:00Z,11.0,0,54.4,0.5,23.2,0,0.1,0,10.7
4,5,NPG Yorkshire,Yorkshire,148,moderate,2024-07-10T22:30Z,2024-07-10T23:00Z,36.5,0,13.8,26.2,7.7,0,0.0,0,15.8
5,6,SP Manweb,North Wales & Merseyside,15,very low,2024-07-10T22:30Z,2024-07-10T23:00Z,1.7,0,3.9,3.2,30.0,0,5.2,0,56.0
6,7,WPD South Wales,South Wales,267,high,2024-07-10T22:30Z,2024-07-10T23:00Z,1.6,0,2.4,67.2,9.7,0,0.5,0,18.6
7,8,WPD West Midlands,West Midlands,72,low,2024-07-10T22:30Z,2024-07-10T23:00Z,5.4,0,8.3,16.5,35.0,0,1.7,0,33.1
8,9,WPD East Midlands,East Midlands,248,high,2024-07-10T22:30Z,2024-07-10T23:00Z,17.2,0,6.5,57.6,3.6,0,0.0,0,15.1
9,10,UKPN East,East England,86,low,2024-07-10T22:30Z,2024-07-10T23:00Z,0.0,0,0.0,21.9,31.0,0,0.0,0,47.1


In [44]:
def solar_generation_live_df(solar_regions_list, extra_fields=True, printing=True):
    """
        Calls an api endpoint per region in solar_regions_list (14 regions)
        solar regions should correspond to those listed at:
        https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes_list
    """
    base_url = f"https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/" #/4 for region_id 4

    #extra fields ctrl-f: Aggregated by PES region
    #doc: https://docs.google.com/document/d/e/2PACX-1vSDFb-6dJ2kIFZnsl-pBQvcH4inNQCA4lYL9cwo80bEHQeTK8fONLOgDf6Wm4ze_fxonqK3EVBVoAIz/pub
    extra_fields = ",".join([
        "bias_error", #estimate of Mean Normalised Bias Error
        "capacity_mwp", # estimate of total effective capacity
        "installedcapacity_mwp", #estimate of installed capacity Megawatt peak
        "lcl_mw", #lower confidence limit 90%
        "stats_error", #estimate of Mean Absolute Percent Error
        "ucl_mw", #upper cofidence limit (90%)
        "uncertainty_MW", #estimate of RMSE * effective capacity
        "site_count", #number of sites
        "updated_gmt", #latest update time of estimate
    ])
    if printing:
        print(f"Fetching solar API regional data from base url: {base_url}<region_id>")
        print("Extra fields:\n- ", "\n- ".join(extra_fields.split(',')) )
        print("Sample API url: ", f"{base_url}{solar_regions_list.iloc[0]}?extra_fields={extra_fields}")
        print(f"Fetching {len(solar_regions_list)} solar regions...")

    regions_solar_data = []
    for api_region_id in  solar_regions_list:
        region_url = base_url + str(api_region_id) + f"?extra_fields={extra_fields}"
        if printing:
            print(f"Fetching live solar data for pes region:", api_region_id)
        response=requests.get(region_url)

        if response.status_code == 200:
            region_data = response.json()
            region_dict = {f"solar_{k}":v for k,v in zip(region_data['meta'], region_data['data'][0])}
            regions_solar_data.append(region_dict)
        else:
            print("response not 200!", response.content)

    solar_live_data_df = pd.DataFrame(regions_solar_data)

    cols_rename = {
        # "datetime_gmt" : "solar_datetime_gmt",
        "solar_pes_id" : "solar_region_id",
        "solar_updated_gmt" : "solar_last_updated_gmt"
    }

    return solar_live_data_df.rename(columns=cols_rename)
    
solar_regions_list = local_reg_geojson.sort_values(by="pes_id")["pes_id"]
solar_df = solar_generation_live_df(solar_regions_list)
solar_df

Fetching solar API regional data from base url: https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/<region_id>
Extra fields:
-  bias_error
- capacity_mwp
- installedcapacity_mwp
- lcl_mw
- stats_error
- ucl_mw
- uncertainty_MW
- site_count
- updated_gmt
Sample API url:  https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/10?extra_fields=bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
Fetching 14 solar regions...
Fetching live solar data for pes region: 10
Fetching live solar data for pes region: 11
Fetching live solar data for pes region: 12
Fetching live solar data for pes region: 13
Fetching live solar data for pes region: 14
Fetching live solar data for pes region: 15
Fetching live solar data for pes region: 16
Fetching live solar data for pes region: 17
Fetching live solar data for pes region: 18
Fetching live solar data for pes region: 19
Fetching live solar data for pes region: 20
Fetching live solar data for pes regi

,solar_region_id,solar_datetime_gmt,solar_generation_mw,solar_bias_error,solar_capacity_mwp,solar_installedcapacity_mwp,solar_lcl_mw,solar_stats_error,solar_ucl_mw,solar_uncertainty_MW,solar_site_count,solar_last_updated_gmt
0,10,2024-07-11T00:30:00Z,0.0,None,2603.144410,2774.729716,None,None,None,None,None,2024-07-11T00:40:26Z
1,11,2024-07-11T00:30:00Z,0.0,None,2295.355024,2435.809979,None,None,None,None,None,2024-07-11T00:40:26Z
2,12,2024-07-11T00:30:00Z,0.0,None,150.569765,159.704710,None,None,None,None,None,2024-07-11T00:40:26Z
3,13,2024-07-11T00:30:00Z,0.0,None,712.477254,751.412779,None,None,None,None,None,2024-07-11T00:40:26Z
4,14,2024-07-11T00:30:00Z,0.0,None,1102.396416,1168.683951,None,None,None,None,None,2024-07-11T00:40:26Z
5,15,2024-07-11T00:30:00Z,0.0,None,504.114969,534.648572,None,None,None,None,None,2024-07-11T00:40:26Z
6,16,2024-07-11T00:30:00Z,0.0,None,555.172080,590.916790,None,None,None,None,None,2024-07-11T00:40:26Z
7,17,2024-07-11T00:30:00Z,0.0,None,265.490704,280.809404,None,None,None,None,None,2024-07-11T00:40:26Z
8,18,2024-07-11T00:30:00Z,0.0,None,369.510047,388.776363,None,None,None,None,None,2024-07-11T00:40:26Z
9,19,2024-07-11T00:30:00Z,0.0,None,879.058465,938.362135,None,None,None,None,None,2024-07-11T00:40:26Z


## function that returns both carbon and solar

This is where the magic happends.

In [9]:
def get_carbon_and_solar_geodf():
    uk_energy_regions_df = get_regional_geojson()
    solar_regions_list = uk_energy_regions_df.sort_values(by="pes_id")["pes_id"]
    
    carbon_df = carbon_intensity_live_df()
    solar_df = solar_generation_live_df(solar_regions_list, extra_fields=True, printing=False)

    out_df = uk_energy_regions_df.merge(carbon_df, how="inner", left_on = "ID", right_on="carbon_region_id")\
                .merge(solar_df, how="inner", left_on="pes_id", right_on="solar_region_id")

    #choose columns
    
    return out_df
get_carbon_and_solar_geodf()

loading uk energy regions from file /home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api/../data/geo/uk_dno_regions_2024_lonlat.geojson
Getting current regional carbon intensity from:  https://api.carbonintensity.org.uk/regional
Converting carbon intensity api response to df...
Fetchings solar regions. Sample API url:  https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/10?extra_fields=bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt


,ID,Name,DNO,Region,DNO_Full,pes_id,geometry,carbon_region_id,dno_region,api_name,...,generation_mw,bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
0,1,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,17,"MULTIPOLYGON (((-5.1162 55.2412, -5.11948 55.2...",1,Scottish Hydro Electric Power Distribution,North Scotland,...,0.0,None,265.496807,280.809404,None,None,None,None,None,2024-07-10T23:10:27Z
1,2,_N,SPEN,South Scotland,SP Energy Networks,18,"MULTIPOLYGON (((-4.7016 54.66506, -4.70051 54....",2,SP Distribution,South Scotland,...,0.0,None,369.518499,388.776363,None,None,None,None,None,2024-07-10T23:10:27Z
2,3,_G,ENWL,North West England,Electricity North West,16,"MULTIPOLYGON (((-2.66161 55.14636, -2.65693 55...",3,Electricity North West,North West England,...,0.0,None,555.184921,590.916790,None,None,None,None,None,2024-07-10T23:10:27Z
3,4,_F,NPG,North East England,Northern Powergrid,15,"MULTIPOLYGON (((-1.59801 55.62683, -1.59862 55...",4,NPG North East,North East England,...,0.0,None,504.126593,534.648572,None,None,None,None,None,2024-07-10T23:10:27Z
4,5,_M,NPG,Yorkshire,Northern Powergrid,23,"MULTIPOLYGON (((-2.22088 54.25104, -2.21629 54...",5,NPG Yorkshire,Yorkshire,...,0.0,None,787.786031,834.404513,None,None,None,None,None,2024-07-10T23:10:27Z
5,6,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,13,"MULTIPOLYGON (((-4.255 53.38794, -4.25559 53.3...",6,SP Manweb,North Wales & Merseyside,...,0.0,None,712.493600,751.412779,None,None,None,None,None,2024-07-10T23:10:27Z
6,7,_K,NGED,South Wales,National Grid Electricity Distribution,21,"MULTIPOLYGON (((-3.11356 51.3757, -3.11456 51....",7,WPD South Wales,South Wales,...,0.0,None,1025.003849,1094.425501,None,None,None,None,None,2024-07-10T23:10:27Z
7,8,_E,NGED,West Midlands,National Grid Electricity Distribution,14,"MULTIPOLYGON (((-1.959 53.21662, -1.95676 53.2...",8,WPD West Midlands,West Midlands,...,0.0,None,1102.421809,1168.683951,None,None,None,None,None,2024-07-10T23:10:27Z
8,9,_B,NGED,East Midlands,National Grid Electricity Distribution,11,"MULTIPOLYGON (((0.26687 52.8124, 0.26697 52.81...",9,WPD East Midlands,East Midlands,...,0.0,None,2295.407969,2435.809979,None,None,None,None,None,2024-07-10T23:10:27Z
9,10,_A,UKPN,East England,UK Power Networks,10,"MULTIPOLYGON (((0.8747 51.51298, 0.87653 51.51...",10,UKPN East,East England,...,0.0,None,2603.204701,2774.729716,None,None,None,None,None,2024-07-10T23:10:27Z


### functions that return a geodf with only either carbon or solar

Likely will go unused unless one whole API goes down?

In [27]:
def get_carbon_intensity_live_only():
    #firstly get the geopandas with regions, to be completed with live data

    uk_regions = get_regional_geojson()

    #get the live data
    carbon_df = carbon_intensity_live_df()

    # cols_to_drop = ["Name", "DNO","DNO_Full", "api_name", "dno_region"]
    #send almost all columns through since we're  getting carbon only
    cols_to_drop = ["api_name"]

    #merge them on their respective id columns

    out_df = pd.merge(left=uk_regions, right=carbon_df, how="inner",\
                      left_on="ID", right_on="carbon_region_id")
    
    return out_df.drop(columns=cols_to_drop)
carbon_geo_df = get_carbon_intensity_live_only()
carbon_geo_df

loading uk energy regions from file /home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api/../data/geo/uk_dno_regions_2024_lonlat.geojson
Getting current regional carbon intensity from:  https://api.carbonintensity.org.uk/regional
Converting carbon intensity api response to df...


,ID,Name,DNO,Region,DNO_Full,pes_id,geometry,carbon_region_id,dno_region,carbon_intensity_forecast,...,carbon_time_to,biomass_perc,coal_perc,imports_perc,gas_perc,nuclear_perc,other_perc,hydro_perc,solar_perc,wind_perc
0,1,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,17,"MULTIPOLYGON (((-5.1162 55.2412, -5.11948 55.2...",1,Scottish Hydro Electric Power Distribution,0,...,2024-07-10T23:30Z,0.0,0,0.0,0.0,0.0,0,2.0,0,98.0
1,2,_N,SPEN,South Scotland,SP Energy Networks,18,"MULTIPOLYGON (((-4.7016 54.66506, -4.70051 54....",2,SP Distribution,16,...,2024-07-10T23:30Z,2.4,0,0.0,3.5,27.7,0,1.0,0,65.4
2,3,_G,ENWL,North West England,Electricity North West,16,"MULTIPOLYGON (((-2.66161 55.14636, -2.65693 55...",3,Electricity North West,28,...,2024-07-10T23:30Z,2.6,0,11.1,6.2,56.3,0,0.3,0,23.6
3,4,_F,NPG,North East England,Northern Powergrid,15,"MULTIPOLYGON (((-1.59801 55.62683, -1.59862 55...",4,NPG North East,17,...,2024-07-10T23:30Z,10.7,0,56.4,0.4,23.5,0,0.1,0,9.0
4,5,_M,NPG,Yorkshire,Northern Powergrid,23,"MULTIPOLYGON (((-2.22088 54.25104, -2.21629 54...",5,NPG Yorkshire,139,...,2024-07-10T23:30Z,42.0,0,13.3,22.2,8.4,0,0.0,0,14.1
5,6,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,13,"MULTIPOLYGON (((-4.255 53.38794, -4.25559 53.3...",6,SP Manweb,16,...,2024-07-10T23:30Z,1.7,0,4.2,3.6,31.0,0,5.3,0,54.1
6,7,_K,NGED,South Wales,National Grid Electricity Distribution,21,"MULTIPOLYGON (((-3.11356 51.3757, -3.11456 51....",7,WPD South Wales,235,...,2024-07-10T23:30Z,2.5,0,3.1,58.8,13.5,0,0.7,0,21.4
7,8,_E,NGED,West Midlands,National Grid Electricity Distribution,14,"MULTIPOLYGON (((-1.959 53.21662, -1.95676 53.2...",8,WPD West Midlands,64,...,2024-07-10T23:30Z,6.4,0,8.1,14.2,36.0,0,1.8,0,33.4
8,9,_B,NGED,East Midlands,National Grid Electricity Distribution,11,"MULTIPOLYGON (((0.26687 52.8124, 0.26697 52.81...",9,WPD East Midlands,224,...,2024-07-10T23:30Z,23.0,0,7.3,49.7,4.5,0,0.0,0,15.6
9,10,_A,UKPN,East England,UK Power Networks,10,"MULTIPOLYGON (((0.8747 51.51298, 0.87653 51.51...",10,UKPN East,76,...,2024-07-10T23:30Z,0.0,0,0.0,19.2,34.8,0,0.0,0,46.0


In [26]:
def get_solar_generation_live_only():
    #contains mapping from id number to id letter (name)
    uk_regions_df = get_regional_geojson()

    solar_regions_list = uk_regions_df.sort_values(by="pes_id")["pes_id"]
    
    #contains live solar data indexed by id number (pes_id)
    solar_df = solar_generation_live_df(solar_regions_list, printing=True)
    
    out_df = pd.merge(left=uk_regions_df, right=solar_df,
                      left_on="pes_id", right_on="solar_region_id")
    return out_df
    
get_solar_generation_live_only()

loading uk energy regions from file /home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api/../data/geo/uk_dno_regions_2024_lonlat.geojson
Fetching solar API regional data from base urlhttps://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/<region_id>
Fetchings solar regions. Sample API url:  https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/10?extra_fields=bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
Extra fields:
 bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
Fetching live solar data for pes region: 10
Fetching live solar data for pes region: 11
Fetching live solar data for pes region: 12
Fetching live solar data for pes region: 13
Fetching live solar data for pes region: 14
Fetching live solar data for pes region: 15
Fetching live solar data for pes region: 16
Fetching live solar data for pes region: 17
Fetching live solar data for pes region: 18
Fetchin

,ID,Name,DNO,Region,DNO_Full,pes_id,geometry,solar_region_id,solar_datetime_gmt,generation_mw,bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,solar_last_updated_gmt
0,1,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,17,"MULTIPOLYGON (((-5.1162 55.2412, -5.11948 55.2...",17,2024-07-10T23:30:00Z,0.0,None,265.496805,280.809404,None,None,None,None,None,2024-07-10T23:40:26Z
1,2,_N,SPEN,South Scotland,SP Energy Networks,18,"MULTIPOLYGON (((-4.7016 54.66506, -4.70051 54....",18,2024-07-10T23:30:00Z,0.0,None,369.518498,388.776363,None,None,None,None,None,2024-07-10T23:40:26Z
2,3,_G,ENWL,North West England,Electricity North West,16,"MULTIPOLYGON (((-2.66161 55.14636, -2.65693 55...",16,2024-07-10T23:30:00Z,0.0,None,555.184918,590.916790,None,None,None,None,None,2024-07-10T23:40:26Z
3,4,_F,NPG,North East England,Northern Powergrid,15,"MULTIPOLYGON (((-1.59801 55.62683, -1.59862 55...",15,2024-07-10T23:30:00Z,0.0,None,504.126591,534.648572,None,None,None,None,None,2024-07-10T23:40:26Z
4,5,_M,NPG,Yorkshire,Northern Powergrid,23,"MULTIPOLYGON (((-2.22088 54.25104, -2.21629 54...",23,2024-07-10T23:30:00Z,0.0,None,787.786030,834.404513,None,None,None,None,None,2024-07-10T23:40:26Z
5,6,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,13,"MULTIPOLYGON (((-4.255 53.38794, -4.25559 53.3...",13,2024-07-10T23:30:00Z,0.0,None,712.493600,751.412779,None,None,None,None,None,2024-07-10T23:40:26Z
6,7,_K,NGED,South Wales,National Grid Electricity Distribution,21,"MULTIPOLYGON (((-3.11356 51.3757, -3.11456 51....",21,2024-07-10T23:30:00Z,0.0,None,1025.003849,1094.425501,None,None,None,None,None,2024-07-10T23:40:26Z
7,8,_E,NGED,West Midlands,National Grid Electricity Distribution,14,"MULTIPOLYGON (((-1.959 53.21662, -1.95676 53.2...",14,2024-07-10T23:30:00Z,0.0,None,1102.421807,1168.683951,None,None,None,None,None,2024-07-10T23:40:26Z
8,9,_B,NGED,East Midlands,National Grid Electricity Distribution,11,"MULTIPOLYGON (((0.26687 52.8124, 0.26697 52.81...",11,2024-07-10T23:30:00Z,0.0,None,2295.407957,2435.809979,None,None,None,None,None,2024-07-10T23:40:26Z
9,10,_A,UKPN,East England,UK Power Networks,10,"MULTIPOLYGON (((0.8747 51.51298, 0.87653 51.51...",10,2024-07-10T23:30:00Z,0.0,None,2603.204698,2774.729716,None,None,None,None,None,2024-07-10T23:40:26Z


In [24]:
def solar_generation_live_geodict():
    solar_geo_df = get_solar_generation_live_only()
    print("returning as geodict")
    return solar_geo_df.to_geo_dict(drop_id=True)
solar_geo = solar_generation_live_geodict()

loading uk energy regions from file /home/louis/code/JammyNinja/Sandbox/energy/energeo_api/api/../data/geo/uk_dno_regions_2024_lonlat.geojson
Fetching solar API regional data from base urlhttps://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/<region_id>
Fetchings solar regions. Sample API url:  https://api.solar.sheffield.ac.uk/pvlive/api/v4/pes/10?extra_fields=bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
Extra fields:
 bias_error,capacity_mwp,installedcapacity_mwp,lcl_mw,stats_error,ucl_mw,uncertainty_MW,site_count,updated_gmt
Fetching live solar data for pes region: 10
Fetching live solar data for pes region: 11
Fetching live solar data for pes region: 12
Fetching live solar data for pes region: 13
Fetching live solar data for pes region: 14
Fetching live solar data for pes region: 15
Fetching live solar data for pes region: 16
Fetching live solar data for pes region: 17
Fetching live solar data for pes region: 18
Fetchin

In [25]:
# solar_geo

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'ID': 1,
    'Name': '_P',
    'DNO': 'SSEN',
    'Region': 'North Scotland',
    'DNO_Full': 'Scottish and Southern Electricity Networks',
    'pes_id': 17,
    'solar_region_id': 17,
    'solar_datetime_gmt': '2024-07-10T23:30:00Z',
    'generation_mw': 0.0,
    'bias_error': None,
    'capacity_mwp': 265.4968045009999,
    'installedcapacity_mwp': 280.80940366400006,
    'lcl_mw': None,
    'stats_error': None,
    'ucl_mw': None,
    'uncertainty_MW': None,
    'site_count': None,
    'solar_last_updated_gmt': '2024-07-10T23:40:26Z'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [(((-5.116204686421463, 55.24120486238761),
       (-5.119483782994426, 55.2413521612087),
       (-5.122969408419094, 55.241865783419996),
       (-5.126032519492363, 55.24262376782178),
       (-5.128565351438425, 55.2436165543859),
       (-5.130301749528905, 55.24476860702587),
       (-5.131468

## Checking local_process_geojson had adapted to the changes

In [30]:
path_to_geodata =  os.path.join("..","data", "geo")
! ls {path_to_geodata}

carbon_intensity_regional.png
national_grid_dno_regions_2024.geojson
national_grid_dno_regions_2024.geojson:Zone.Identifier
test_uk_dno_regions_2024_lonlat.geojson
uk_dno_regions_2024_lonlat.geojson
uk_dno_regions_lonlat.geojson


In [32]:
carbon_intensity_live_df()

Getting current regional carbon intensity from:  https://api.carbonintensity.org.uk/regional
Converting carbon intensity api response to df...


,carbon_region_id,dno_region,api_name,carbon_intensity_forecast,carbon_intensity_index,carbon_time_from,carbon_time_to,biomass_perc,coal_perc,imports_perc,gas_perc,nuclear_perc,other_perc,hydro_perc,solar_perc,wind_perc
0,1,Scottish Hydro Electric Power Distribution,North Scotland,0,very low,2024-07-10T23:30Z,2024-07-11T00:00Z,0.0,0,0.0,0.0,0.0,0,1.9,0,98.1
1,2,SP Distribution,South Scotland,15,very low,2024-07-10T23:30Z,2024-07-11T00:00Z,2.4,0,0.0,3.0,29.5,0,1.0,0,64.1
2,3,Electricity North West,North West England,25,very low,2024-07-10T23:30Z,2024-07-11T00:00Z,2.8,0,10.9,5.4,56.0,0,0.3,0,24.7
3,4,NPG North East,North East England,17,very low,2024-07-10T23:30Z,2024-07-11T00:00Z,10.9,0,54.6,0.4,23.2,0,0.1,0,10.8
4,5,NPG Yorkshire,Yorkshire,123,moderate,2024-07-10T23:30Z,2024-07-11T00:00Z,37.2,0,16.8,19.7,9.9,0,0.1,0,16.4
5,6,SP Manweb,North Wales & Merseyside,14,very low,2024-07-10T23:30Z,2024-07-11T00:00Z,1.8,0,4.0,2.9,31.4,0,5.7,0,54.2
6,7,WPD South Wales,South Wales,214,high,2024-07-10T23:30Z,2024-07-11T00:00Z,2.2,0,3.7,53.6,15.9,0,0.9,0,23.8
7,8,WPD West Midlands,West Midlands,56,low,2024-07-10T23:30Z,2024-07-11T00:00Z,5.4,0,8.8,12.3,36.6,0,2.0,0,34.9
8,9,WPD East Midlands,East Midlands,213,high,2024-07-10T23:30Z,2024-07-11T00:00Z,20.0,0,9.0,47.9,6.1,0,0.0,0,17.0
9,10,UKPN East,East England,66,low,2024-07-10T23:30Z,2024-07-11T00:00Z,0.0,0,0.0,16.7,37.2,0,0.0,0,46.0


In [39]:
#helper for geojson processor
def convert_geodf_to_lonlat(geodf, lonlat_EPSG = 4326):
    """ projects the co-ordinates to lonlat
        GeoPandas projections
        https://geopandas.org/en/stable/docs/user_guide/projections.html
    """
    print("Co-ordinate Reference System before: ", geodf.crs)
    geodf_lonlat = geodf.to_crs(epsg=lonlat_EPSG)
    print("Co-ordinate Reference System after: ", geodf_lonlat.crs)
    return geodf_lonlat

In [42]:
def process_local_geojson(filename_out, filename_in = "national_grid_dno_regions_2024.geojson"):
    """
        rename the IDs from downloaded geojson file with IDs as served by API
        only needs to be run once...
    """
    # filename_out = geo_file_prod
    filepath_in = os.path.join(path_to_geodata, filename_in)
    filepath_out = os.path.join(path_to_geodata, filename_out)
    print(f"processing file: {filepath_in}")

    # Read the GeoJSON file
    uk_regions_2024 = gpd.read_file(filepath_in)

    #rename some areas so we can sort alphabetically and match api region id to region ids from file
    rename_geojson_areas_to_api = {"Southern England":  "South England", "South and Central Scotland" : "South Scotland"}
    uk_regions_2024["Area"] = uk_regions_2024["Area"].map(lambda x: rename_geojson_areas_to_api.get(x,x) )

    #get sample api response to lineup the ids
    region_response_df = carbon_intensity_live_df()

    #line up the ids from file + api
    lineup_file_df = uk_regions_2024.sort_values(by="Area").rename(columns={"ID" : "geojson_id", "Area" : "geojson_name"}).reset_index(drop=True)
    # display(lineup_file_df)
    lineup_api_df = region_response_df.iloc[:14, :3].sort_values(by="api_name").reset_index(drop=True)
    display(lineup_api_df)
    lineup_df = pd.concat([lineup_file_df,lineup_api_df], axis=1)[["geojson_id","carbon_region_id", "geojson_name", "api_name"]]
    #create id mapping dict thanks to lineup_df
    id_map_dict = {}

    for geo_id, geo_name, api_id,api_name in zip(lineup_df.geojson_id, lineup_df.geojson_name, lineup_df.carbon_region_id,lineup_df.api_name):
        print(f"Geo name | Api name: {geo_name} | {api_name} - Geojson id | Api id: {geo_id} -> {api_id}" )
        id_map_dict[geo_id] = [api_id, api_name]

    #store the original ids in new column as will be useful later
    uk_regions_2024["pes_id"] = uk_regions_2024["ID"]

    #execute the mapping of IDs
    uk_regions_2024["ID"] = uk_regions_2024["ID"].map(lambda id : id_map_dict[id][0])

    #rename Area -> Region and sort by ID
    uk_regions_2024 = uk_regions_2024.sort_values(by="ID").rename(columns={"Area" : "Region"})

    #convert co-ordinates to longitude and latitude
    print("Converting co-ordinates to longitude latitude projection")
    uk_regions_2024_lonlat = convert_geodf_to_lonlat(uk_regions_2024)

    print(f"saving processed file to: {filepath_out}")
    uk_regions_2024_lonlat.to_file(filepath_out)
    return uk_regions_2024_lonlat
process_local_geojson(filename_out="test.geojson")

processing file: ../data/geo/national_grid_dno_regions_2024.geojson
Getting current regional carbon intensity from:  https://api.carbonintensity.org.uk/regional
Converting carbon intensity api response to df...


,carbon_region_id,dno_region,api_name
0,10,UKPN East,East England
1,9,WPD East Midlands,East Midlands
2,13,UKPN London,London
3,4,NPG North East,North East England
4,1,Scottish Hydro Electric Power Distribution,North Scotland
5,6,SP Manweb,North Wales & Merseyside
6,3,Electricity North West,North West England
7,14,UKPN South East,South East England
8,12,SSE South,South England
9,2,SP Distribution,South Scotland


Geo name | Api name: East England | East England - Geojson id | Api id: 10 -> 10
Geo name | Api name: East Midlands | East Midlands - Geojson id | Api id: 11 -> 9
Geo name | Api name: London | London - Geojson id | Api id: 12 -> 13
Geo name | Api name: North East England | North East England - Geojson id | Api id: 15 -> 4
Geo name | Api name: North Scotland | North Scotland - Geojson id | Api id: 17 -> 1
Geo name | Api name: North Wales, Merseyside and Cheshire | North Wales & Merseyside - Geojson id | Api id: 13 -> 6
Geo name | Api name: North West England | North West England - Geojson id | Api id: 16 -> 3
Geo name | Api name: South East England | South East England - Geojson id | Api id: 19 -> 14
Geo name | Api name: South England | South England - Geojson id | Api id: 20 -> 12
Geo name | Api name: South Scotland | South Scotland - Geojson id | Api id: 18 -> 2
Geo name | Api name: South Wales | South Wales - Geojson id | Api id: 21 -> 7
Geo name | Api name: South West England | Sout

,ID,Name,DNO,Region,DNO_Full,geometry,pes_id
7,1,_P,SSEN,North Scotland,Scottish and Southern Electricity Networks,"MULTIPOLYGON (((-5.1162 55.2412, -5.11948 55.2...",17
12,2,_N,SPEN,South Scotland,SP Energy Networks,"MULTIPOLYGON (((-4.7016 54.66506, -4.70051 54....",18
6,3,_G,ENWL,North West England,Electricity North West,"MULTIPOLYGON (((-2.66161 55.14636, -2.65693 55...",16
5,4,_F,NPG,North East England,Northern Powergrid,"MULTIPOLYGON (((-1.59801 55.62683, -1.59862 55...",15
11,5,_M,NPG,Yorkshire,Northern Powergrid,"MULTIPOLYGON (((-2.22088 54.25104, -2.21629 54...",23
3,6,_D,SPEN,"North Wales, Merseyside and Cheshire",SP Energy Networks,"MULTIPOLYGON (((-4.255 53.38794, -4.25559 53.3...",13
9,7,_K,NGED,South Wales,National Grid Electricity Distribution,"MULTIPOLYGON (((-3.11356 51.3757, -3.11456 51....",21
4,8,_E,NGED,West Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((-1.959 53.21662, -1.95676 53.2...",14
1,9,_B,NGED,East Midlands,National Grid Electricity Distribution,"MULTIPOLYGON (((0.26687 52.8124, 0.26697 52.81...",11
0,10,_A,UKPN,East England,UK Power Networks,"MULTIPOLYGON (((0.8747 51.51298, 0.87653 51.51...",10
